In [8]:
from qdrant_client import QdrantClient
from pandas import DataFrame
from PIL import Image
import os 
import pandas as pd

# qdrant_client = QdrantClient(
#     url="https://02cfa3da-e5fd-484e-ad1f-b17530cf0530.us-east4-0.gcp.cloud.qdrant.io:6333/", 
#     api_key="ms5CTCc7UA4OnkxqZBp9KAwM2d3h-N6Cex_QVm2Icmvn1enD5IpWAQ",
# )

qdrant_client = QdrantClient(
    url="https://a7d15b9f-649d-4c0f-9433-2a6c06cc66cc.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="mQ3vUPBNtp0mRY6h0gTFMRncal8o1o_trEhck6tYv327wNB7c8WLHw",
)

In [9]:
# import os
# import pandas as pd
# from PIL import Image
# from io import BytesIO
# import math
# import base64
# 
# class ImageDataset:
#     def __init__(self, base_path, num_samples_per_class=50):
#         self.base_path = base_path
#         self.num_samples_per_class = num_samples_per_class
#         self.df = pd.DataFrame()
#         self._load_data()
# 
#     def _load_data(self):
#         for class_id, class_path in enumerate(os.listdir(self.base_path)):
#             class_file = os.listdir(f'{self.base_path}/{class_path}')
#             samples_in_class = len(class_file)
#             samples_in_class = min(samples_in_class, self.num_samples_per_class)
#             
#             train_samples_files = class_file[:samples_in_class]
#             train_samples_files = list(map(lambda x: f'{self.base_path}/{class_path}/{x}', train_samples_files))
#             
#             payload = pd.DataFrame({
#                 'image_path': train_samples_files,
#                 'type': class_path
#             })
#             self.df = pd.concat([self.df, payload])
# 
#     def resize_image(self, image, target_width=256):
#         image_aspect_ratio = image.size[0] / image.size[1]
#         resized_img = image.resize(
#             [target_width, math.floor(target_width * image_aspect_ratio)]
#         )
#         return resized_img
#     
#     def convert_image_to_base64(self, image):
#         buffered = BytesIO()
#         image.save(buffered, format="JPEG")
#         return base64.b64encode(buffered.getvalue()).decode("utf-8")
# 
#     def __getitem__(self, idx):
#         if idx >= len(self.df):
#             raise IndexError("Index out of range")
#         
#         row = self.df.iloc[idx]
#         image = Image.open(row['image_path'])
#         image = self.resize_image(image)
#         encoded_image = self.convert_image_to_base64(image)
#         
#         return image, row['type'], encoded_image
# 
#     def __len__(self):
#         return len(self.df)
# 
# # Example usage:
# TRAIN_BASE_PATH = '../afhq/train'
# NUM_SAMPLE_FOR_CLASS = 50
# 
# data = ImageDataset(TRAIN_BASE_PATH, NUM_SAMPLE_FOR_CLASS)

In [10]:
# df_origin = pd.read_excel('../Data/instagram_posts.xlsx')
# df_origin['post_date'] = df_origin['post_date'].dt.strftime('%Y-%m-%d_%H-%M-%S')
# df_origin['post_date']

In [11]:
# train_files = os.listdir('../Data/instagram_photo')
# df_matching = DataFrame({
#     'instagram_user': [],
#     'image_path': [],
#     'link': [],
# })
# matching = []
# for i in range(len(df_origin)):
#     post_date = df_origin['post_date'][i]
#     for file in train_files:
#         if file.endswith('.jpg'):
#             if post_date == file[:19]:
#                 image_path = f'../Data/instagram_photo/{file}'
#                 post_date = df_origin['post_date'][i]
#                 instagram_user = df_origin['username'][i]
#                 link = df_origin['post_url'][i]
#     
#                 matching.append({
#                     'instagram_user': instagram_user,
#                     'image_path': image_path,
#                     'link': link,
#                 })
# 
# df_matching = pd.DataFrame(matching)
# df_matching

In [41]:
import os
import pandas as pd
from io import BytesIO
import math
import base64
from transformers import AutoImageProcessor, ResNetForImageClassification
from qdrant_client.models import VectorParams, Distance, PointStruct

class InstagramPhotoMatcher:
    def __init__(self, data_path, origin_df):
        self.data_path = data_path
        self.origin_df = origin_df
        self.train_files = os.listdir(data_path)
        self.df_matching = self.create_matching_dataframe()
        
        self.processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
        self.model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
        
        self.collection_name = 'instagram_data'
        # self.collection = qdrant_client.create_collection(
        #     collection_name=self.collection_name,
        #     vectors_config=VectorParams(
        #         size=1000,
        #         distance=Distance.COSINE
        #     ),
        # )

    def create_matching_dataframe(self):
        matching = []
        for i in range(len(self.origin_df)):
            post_date = self.origin_df['post_date'][i]
            for file in self.train_files:
                if file.endswith('.jpg') and post_date == file[:19]:
                    image_path = os.path.join(self.data_path, file)
                    instagram_user = self.origin_df['username'][i]
                    full_name = self.origin_df['full_name'][i]
                    bio = self.origin_df['biography'][i]
                    caption = self.origin_df['post_caption'][i]
                    link = self.origin_df['post_url'][i]
                    matching.append({
                        'instagram_user': instagram_user,
                        'image_path': image_path.replace("\\", '/'),
                        'link': link,
                        'full_name': full_name,
                        'bio': bio,
                        'caption': caption,
                    })
        return pd.DataFrame(matching)
    
    def resize_image(self, image, target_width=256):
        image_aspect_ratio = image.size[0] / image.size[1]
        resized_img = image.resize(
            [target_width, math.floor(target_width * image_aspect_ratio)]
        )
        return resized_img

    def convert_image_to_base64(self, image):
        buffered = BytesIO()
        image.save(buffered, format="JPEG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")

    def upload_item(self, idx):
        if idx >= len(self.df_matching):
            raise IndexError("Index out of range")
        
        item_data = self.__getitem__(idx)
        row = item_data['row']
        encoded_image = item_data['encoded_image']
        logits = item_data['logits']
    
        bio = row['bio'][:512] if isinstance(row['bio'], str) else ''
        
        points = [
            PointStruct(
                id=idx,
                payload={
                    'instagram_user': row['instagram_user'],
                    'image_path': row['image_path'],
                    'link': row['link'],
                    'full_name': row['full_name'],
                    'bio': bio,
                    'encoded_image': encoded_image,
                },
                vector=logits.flatten().tolist()
            )
        ]
        
        print(qdrant_client.upsert(
            collection_name=self.collection_name,
            points=points
        ))


    def __getitem__(self, idx):
        if idx >= len(self.df_matching):
            raise IndexError("Index out of range")
        
        row = self.df_matching.iloc[idx]
        image = Image.open(row['image_path'])
        image = image.convert('RGB')
        image = self.resize_image(image)
        encoded_image = self.convert_image_to_base64(image)
        
        inputs = self.processor(images=image, return_tensors="pt")
        outputs = self.model(**inputs)
        
        return {
            'row': row.to_dict(),
            'image': image,
            'encoded_image': encoded_image,
            'logits': outputs.logits
        }

    
    def __len__(self):
        return len(self.df_matching)
        
data_path = '../Data/instagram_photo'
df_origin = pd.read_excel('../Data/instagram_posts.xlsx')
df_origin['post_date'] = df_origin['post_date'].dt.strftime('%Y-%m-%d_%H-%M-%S')
matcher = InstagramPhotoMatcher(data_path, df_origin)

In [29]:
# import os
# import pandas as pd
# from io import BytesIO
# import math
# import base64
# from transformers import AutoImageProcessor, ResNetForImageClassification
# from qdrant_client.models import VectorParams, Distance, PointStruct
# 
# class InstagramPhotoMatcher:
#     def __init__(self, data_path, origin_df):
#         self.data_path = data_path
#         self.origin_df = origin_df
#         self.train_files = os.listdir(data_path)
#         self.df_matching = self.create_matching_dataframe()
#         
#         self.processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
#         self.model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
#         
#         self.collection_name = 'instagram_data'
#         # self.collection = qdrant_client.create_collection(
#         #     collection_name=self.collection_name,
#         #     vectors_config=VectorParams(
#         #         size=1000,
#         #         distance=Distance.COSINE
#         #     ),
#         # )
# 
#     def create_matching_dataframe(self):
#         matching = []
#         for i in range(len(self.origin_df)):
#             post_date = self.origin_df['post_date'][i]
#             for file in self.train_files:
#                 if file.endswith('.jpg') and post_date == file[:19]:
#                     image_path = os.path.join(self.data_path, file)
#                     instagram_user = self.origin_df['username'][i]
#                     full_name = self.origin_df['full_name'][i]
#                     bio = self.origin_df['biography'][i]
#                     caption = self.origin_df['post_caption'][i]
#                     link = self.origin_df['post_url'][i]
#                     matching.append({
#                         'instagram_user': instagram_user,
#                         'image_path': image_path.replace("\\", '/'),
#                         'link': link,
#                         'full_name': full_name,
#                         'bio': bio,
#                         'caption': caption,
#                     })
#         return pd.DataFrame(matching)
#     
#     def resize_image(self, image, target_width=256):
#         image_aspect_ratio = image.size[0] / image.size[1]
#         resized_img = image.resize(
#             [target_width, math.floor(target_width * image_aspect_ratio)]
#         )
#         return resized_img
# 
#     def convert_image_to_base64(self, image):
#         buffered = BytesIO()
#         image.save(buffered, format="JPEG")
#         return base64.b64encode(buffered.getvalue()).decode("utf-8")
# 
#     def __getitem__(self, idx):
#         if idx >= len(self.df_matching):
#             raise IndexError("Index out of range")
#         
#         row = self.df_matching.iloc[idx]
#         image = Image.open(row['image_path'])
#         image = image.convert('RGB')
#         image = self.resize_image(image)
#         encoded_image = self.convert_image_to_base64(image)
#         
#         inputs = self.processor(images=image, return_tensors="pt")
#         outputs = self.model(**inputs)
#         
#         # points = [
#         #     models.PointStruct(
#         #         id=idx,
#         #         payload=data_dict[idx],
#         #         vector=embeddings[idx].flatten().tolist()
#         #     )
#         #     for idx, _ in enumerate(data_dict)
#         # ]
#         
#         # convert bio to string and only take the first 512 characters
#         if isinstance(row['bio'], str):
#             bio = row['bio'][:512]
#         else:
#             bio = ''
#         
#         
#         points = [
#             PointStruct(
#                 id=idx,
#                 payload={
#                     'instagram_user': row['instagram_user'],
#                     'image_path': row['image_path'],
#                     'link': row['link'],
#                     'full_name': row['full_name'],
#                     'bio': bio,
#                     'encoded_image': encoded_image,
#                 },
#                 vector=outputs.logits.flatten().tolist()
#             )
#         ]
#         
#         qdrant_client.upsert(
#             collection_name=self.collection_name,
#             points=points
#         )
#         # data_dict[i] = {
#         #     'instagram_user': matcher[i][0]['instagram_user'],
#         #     'image_path': matcher[i][0]['image_path'],
#         #     'link': matcher[i][0]['link'],
#         #     'full_name': matcher[i][0]['full_name'],
#         #     'bio': matcher[i][0]['bio'],
#         #     'encoded_image': matcher[i][2],
#         # }
#         
#         return self.df_matching.iloc[idx].to_dict(), image, encoded_image, outputs.logits
#     
#     def __len__(self):
#         return len(self.df_matching)
#         
# # Usage
# # df_origin = pd.DataFrame({...})  # Assuming df_origin is already defined
# data_path = '../Data/instagram_photo'
# df_origin = pd.read_excel('../Data/instagram_posts.xlsx')
# df_origin['post_date'] = df_origin['post_date'].dt.strftime('%Y-%m-%d_%H-%M-%S')
# matcher = InstagramPhotoMatcher(data_path, df_origin)

In [30]:
# for i in range(1852, len(matcher)):
#     matcher[i]

In [6]:
# from io import BytesIO
# import math
# import base64
# 
# target_width = 128
# def resize_image(image_path):
#     pil_img = Image.open(image_path)
#     image_aspect_ratio = pil_img.size[0] / pil_img.size[1]
#     resized_img = pil_img.resize(
#         [target_width, math.floor(target_width * image_aspect_ratio)]
#     )
#     return resized_img
# 
# def convert_image_to_base64(image):
#     buffered = BytesIO()
#     image.save(buffered, format="JPEG")
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")
# 
# resize_images = list(map( lambda x: resize_image(x), df_matching['image_path']))
# base64_str = list(map( lambda x: convert_image_to_base64(x), resize_images))
# 
# # resized_images = list(map( lambda x: resize_image(x), train_samples_files))
# # base64_str = list(map( lambda x: convert_image_to_base64(x), resized_images))
# # payload['image'] = base64_str

In [7]:
# TRAIN_BASE_PATH = '../afhq/train'
# NUM_SAMPLE_FOR_CLASS = 50
# 
# df = pd.DataFrame()
# for class_id, class_path in enumerate(os.listdir(TRAIN_BASE_PATH)):
#     class_file = os.listdir(f'{TRAIN_BASE_PATH}/{class_path}')
#     samples_in_class = len(class_file)
#     samples_in_class = min(samples_in_class, NUM_SAMPLE_FOR_CLASS)
#     
#     train_samples_files = class_file[:samples_in_class]
#     train_samples_files = list(map(lambda x: f'{TRAIN_BASE_PATH}/{class_path}/{x}', train_samples_files))
#     train_samples_images = list(map(lambda x: Image.open(x), train_samples_files))
#     
#     payload = DataFrame.from_records({
#         'image_path': train_samples_files,
#     })
#     payload['type'] = class_path
#     df = pd.concat([df, payload])
# df

In [8]:
# import os
# 
# train_files = os.listdir('../afhq/train/cat')
# 
# train_samples_files = train_files[:200]
# train_samples_files = list(map(lambda x: f'../afhq/train/cat/{x}', train_samples_files))
# train_samples_images = list(map(lambda x: Image.open(x), train_samples_files))

In [9]:
# payload = DataFrame.from_records({
#         'image_path': train_samples_files,
#     })
# payload['type'] = 'cat'
# payload

In [10]:
# from io import BytesIO
# import math
# import base64
# 
# target_width = 256
# def resize_image(image_path):
#     pil_img = Image.open(image_path)
#     image_aspect_ratio = pil_img.size[0] / pil_img.size[1]
#     resized_img = pil_img.resize(
#         [target_width, math.floor(target_width * image_aspect_ratio)]
#     )
#     return resized_img
# 
# def convert_image_to_base64(image):
#     buffered = BytesIO()
#     image.save(buffered, format="JPEG")
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")
# 
# 
# resized_images = list(map( lambda x: resize_image(x), train_samples_files))
# base64_str = list(map( lambda x: convert_image_to_base64(x), resized_images))
# payload['image'] = base64_str

In [11]:
# payload

In [12]:
# resized_images = []
# for i in range(len(data)):
#     resized_image, _, _ = data[i]
#     resized_images.append(resized_image)
# resized_images

In [13]:
# len(matcher)

In [14]:
# resized_images = []
# for i in range(50):
#     _, image, _ = matcher[i]
#     resized_images.append(image)
#     
# len(resized_images)

In [15]:
# from transformers import AutoImageProcessor, ResNetForImageClassification
# 
# processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
# model = ResNetForImageClassification.from_pretrained("microsoft/resnet-18")
# 
# inputs = processor(images=resized_images, return_tensors="pt")
# outputs = model(**inputs)

In [16]:
# del resized_images
# matcher[0][3].shape # => [1, 1000]

In [17]:
# embeddings = outputs.logits
# embeddings_length = embeddings.shape[1]
# embeddings, embeddings_length

In [22]:
# n_selected = 1000

In [21]:
# from qdrant_client.models import VectorParams, Distance
# 
# collection_name = 'instagram_data'
# collection = qdrant_client.create_collection(
#     collection_name=collection_name,
#     vectors_config=VectorParams(
#         size=matcher[0][3].shape[1],
#         distance=Distance.COSINE
#     ),
# )
# collection

In [32]:
# payload_2 = payload.copy()
# payload_dicts = payload_2.to_dict(orient='records')
# payload_dicts

In [33]:
# from qdrant_client.grpc import PointStruct
# from qdrant_client.grpc.points_pb2 import PointId
# 
# points = []
# for idx, payload_dict in enumerate(payload):
#     point_id = PointId(num=idx)  # Create a PointId object
#     vector = embeddings[idx]
#     payload = payload_dict
# 
#     points.append(PointStruct(id=point_id, vector=vector, payload=payload))
# 
# print(points)  # Now, you should see correctly structured PointStruct instances


In [34]:
# data_dict = {}
# for i in range(len(data)):
#     image, image_type, encoded_image = data[i]
#     data_dict[i] = {'type': image_type, 'image': encoded_image}

In [23]:
# data_dict = {}
# embeddings = []
# for i in range(1000):
#     data_dict[i] = {
#         'instagram_user': matcher[i][0]['instagram_user'],
#         'image_path': matcher[i][0]['image_path'],
#         'link': matcher[i][0]['link'],
#         'full_name': matcher[i][0]['full_name'],
#         'bio': matcher[i][0]['bio'],
#         'encoded_image': matcher[i][2],
#     }
#     embeddings.append(matcher[i][3])

In [43]:
# data_dict[2]

In [44]:
# from qdrant_client import models
# points = [
#     models.PointStruct(
#         id=idx,
#         payload=data_dict[idx],
#         vector=embeddings[idx].flatten().tolist()
#     )
#     for idx, _ in enumerate(data_dict)
# ]

In [45]:
# from qdrant_client import models
# 
# points = [
#     models.PointStruct(
#         id=idx,
#         payload=data_dict[idx],
#         vector=embeddings[idx].tolist()
#     )
#     for idx, _ in enumerate(data_dict)
# ]
# points

In [ ]:
matcher

In [46]:
# qdrant_client.upsert(
#     collection_name=collection_name,
#     points=points
# )

In [ ]:
# qdrant_client.upload_points(
#     collection_name=collection_name,
#     points=points
# )